## 1. Quantization Model




Let an sRGB pixel be $ \mathbf{p} = [R, G, B]^T $ with channel values in $[0, 255]$. Define gamma-space luminance $L = 0.299 R + 0.587 G + 0.114 B$. For a chosen bit depth $b \in \{1,\dots,16\}$, the number of levels is $K = 2^b$ and the step size is $\Delta = 256 / K$.


Quantized luminance $L_q$ is computed by
$$
L_q = \left\lfloor \frac{L}{\Delta} \right\rfloor \Delta.
$$
The bucket index is $k = \lfloor L / \Delta \rfloor \in \{0, \dots, K-1\}$. Error per pixel is
$$
 e_L = L_q - L, \quad |e_L| < \Delta.
$$


We scale the RGB channels by a common factor
$$
f = \frac{L_q}{L + \epsilon},
$$
with $\epsilon$ preventing division by zero. The processed pixel is
$$
\mathbf{p}_q = f \cdot \mathbf{p} \quad \text{clamped to } [0, 255].
$$
This preserves chroma ratios while matching the quantized luminance.


**Code (quantize and bucket assignment):**

```typescript
const reducing = currentBitDepth[0] < imgData.originalBitDepth;
const levels = Math.pow(2, currentBitDepth[0]);
const step = 256 / levels;

for (let i = 0; i < pixels.length; i += 4) {
  const r = pixels[i];
  const g = pixels[i + 1];
  const b = pixels[i + 2];
  const luminosity = 0.299 * r + 0.587 * g + 0.114 * b;
  lumOrig[i / 4] = luminosity;

  if (reducing) {
    const quantized = Math.floor(luminosity / step) * step;
    const bucket = Math.floor(luminosity / step);
    const factor = quantized / luminosity || 0;
    pixels[i] = Math.min(255, r * factor);
    pixels[i + 1] = Math.min(255, g * factor);
    pixels[i + 2] = Math.min(255, b * factor);
    lumProc[i / 4] = 0.299 * pixels[i] + 0.587 * pixels[i + 1] + 0.114 * pixels[i + 2];
    bucketIdx[i / 4] = bucket;
  } else {
    lumProc[i / 4] = luminosity;
    bucketIdx[i / 4] = Math.floor(luminosity / step);
  }
}
```


## 2. Band Edge Detection and Masking

We form luminance arrays for the original $L_{\text{orig}}$ and processed $L_{\text{proc}}$. Sobel magnitude for an image $X$ is
$$
S_X = |G_x| + |G_y|,
$$
with standard 3×3 Sobel kernels. We then take a per-pixel edge lift
$$
D = \max(0, S_{\text{proc}} - S_{\text{orig}}).
$$
When bit depth is not reduced, $L_{\text{proc}} = L_{\text{orig}}$ and $D = 0$.

Bucket boundaries are detected by checking 4-neighbor bucket changes in $k$. A pixel is marked boundary if any neighbor has a different bucket index. The final edge mask is
$$
M = \text{dilate}\big( \mathbf{1}[D \ge \tau] \odot \mathbf{1}[\text{bucket change}] \big),
$$
where $\tau = 0.14 \cdot \max(D)$ and dilation is 3×3. Mask pixels are overlaid with an auto-chosen high-contrast color (yellow, magenta, or cyan).



**Code (Sobel diff and boundary mask):**

```typescript
const sobel = (arr: Float32Array) => { /* 3x3 Sobel, returns {out, maxVal} */ };
const sobelOrigR = sobel(chanOrig[0]);
const sobelOrigG = sobel(chanOrig[1]);
const sobelOrigB = sobel(chanOrig[2]);
const sobelProcR = sobel(chanProc[0]);
const sobelProcG = sobel(chanProc[1]);
const sobelProcB = sobel(chanProc[2]);

const diffMag = new Float32Array(width * height);
let maxDiff = 0;
for (let i = 0; i < diffMag.length; i++) {
  const origMax = Math.max(sobelOrigR.out[i], sobelOrigG.out[i], sobelOrigB.out[i]);
  const procMax = Math.max(sobelProcR.out[i], sobelProcG.out[i], sobelProcB.out[i]);
  const d = Math.max(0, procMax - origMax);
  diffMag[i] = d;
  if (d > maxDiff) maxDiff = d;
}

const bucketMask = new Uint8Array(width * height);
for (let y = 1; y < height - 1; y++) {
  for (let x = 1; x < width - 1; x++) {
    const idx = y * width + x;
    const b = bucketIdx[idx];
    if (bucketIdx[idx - 1] !== b || bucketIdx[idx + 1] !== b || bucketIdx[idx - width] !== b || bucketIdx[idx + width] !== b) {
      bucketMask[idx] = 1;
    }
  }
}

const threshold = maxDiff * 0.2;
const dilated = new Uint8Array(width * height);
for (let y = 1; y < height - 1; y++) {
  for (let x = 1; x < width - 1; x++) {
    const idx = y * width + x;
    if (bucketMask[idx] && diffMag[idx] >= threshold && threshold > 0) {
      for (let dy = -1; dy <= 1; dy++) {
        for (let dx = -1; dx <= 1; dx++) {
          dilated[idx + dy * width + dx] = 1;
        }
      }
    }
  }
}
```

